In [1]:
import torch as tn
from torchvision import datasets, transforms
import torchtt as tntt
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [2]:
device_name = 'cuda:0'
data_dir_test = 'seg_test/'
data_dir_train = 'seg_train/'
N_shape = [15,10]
batch_size = 32

In [3]:
transform_train = transforms.Compose([transforms.Resize(N_shape[0]*N_shape[1]), transforms.CenterCrop(N_shape[0]*N_shape[1]), transforms.ToTensor()]) #, transforms.Normalize(tn.tensor([0.4885, 0.4525, 0.4163]), tn.tensor([0.2549, 0.2476, 0.2495]))])
dataset_train = datasets.ImageFolder(data_dir_train, transform=transform_train)
dataloader_train = tn.utils.data.DataLoader(dataset_train, batch_size=99999, shuffle=True, pin_memory = True, num_workers = 16)

transform_test = transforms.Compose([transforms.Resize(N_shape[0]*N_shape[1]), transforms.CenterCrop(N_shape[0]*N_shape[1]), transforms.ToTensor()]) #, transforms.Normalize(tn.tensor([0.4885, 0.4525, 0.4163]), tn.tensor([0.2549, 0.2476, 0.2495])) ])
dataset_test = datasets.ImageFolder(data_dir_test, transform=transform_test)
dataloader_test = tn.utils.data.DataLoader(dataset_test, batch_size=99999, shuffle=True, pin_memory = True, num_workers = 16)

inputs_train = list(dataloader_train)[0][0]
labels_train = list(dataloader_train)[0][1]
 
inputs_test = list(dataloader_test)[0][0]
labels_test = list(dataloader_test)[0][1]

print('Training set', inputs_train.shape,labels_train.shape)
print('Test set ',inputs_test.shape, labels_test.shape)

Training set torch.Size([14034, 3, 150, 150]) torch.Size([14034])
Test set  torch.Size([3000, 3, 150, 150]) torch.Size([3000])


In [4]:
class BasicTT(nn.Module):
    def __init__(self):
        super().__init__()
        p = 0.25
        self.ttl1 = tntt.nn.LinearLayerTT([3]+N_shape+N_shape, [16]+N_shape+N_shape, [1,9,3,3,2,1], initializer = 'He')
        self.dropout1 = nn.Dropout(p)
        self.ttl2 = tntt.nn.LinearLayerTT([16]+N_shape+N_shape, [16,8,8,8,8], [1,4,2,2,2,1], initializer = 'He')
        self.dropout2 = nn.Dropout(p)
        self.ttl3 = tntt.nn.LinearLayerTT([16,8,8,8,8], [8,4,4,4,4], [1,4,2,2,2,1], initializer = 'He')
        self.dropout3 = nn.Dropout(p)
        self.ttl4 = tntt.nn.LinearLayerTT([8,4,4,4,4], [3,3,3,3,3], [1,2,2,2,2,1], initializer = 'He')
        self.dropout4 = nn.Dropout(p)
        self.ttl5 = tntt.nn.LinearLayerTT([3,3,3,3,3], [3,3,3,3,3], [1,2,2,2,2,1], initializer = 'He')
        self.dropout5 = nn.Dropout(p)
        self.linear = nn.Linear(3**5, 6, dtype = tn.float32)
        self.logsoftmax = nn.LogSoftmax(1)

    def forward(self, x):
        x = self.ttl1(x)
        x = self.dropout1(x)
        x = tn.relu(x)
        x = self.ttl2(x)
        x = self.dropout2(x)
        x = tn.relu(x)
        x = self.ttl3(x)
        x = self.dropout3(x)
        x = tn.relu(x)
        x = self.ttl4(x)
        x = self.dropout4(x)
        x = tn.relu(x)
        x = self.ttl5(x)
        x = self.dropout5(x)
        x = tn.relu(x)
        x = x.view(-1,3**5)
        x = self.linear(x)
        return self.logsoftmax(x)



In [5]:
model = BasicTT()        
model.to(device_name)

optimizer = tn.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
#optimizer = tn.optim.Adam(model.parameters(), lr=0.001)
scheduler = tn.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

loss_function = tn.nn.CrossEntropyLoss()

In [6]:
def do_epoch(i):
    
    loss_total = 0.0
    n_total = 0
    n_correct = 0
    #tme = datetime.datetime.now()
    n_batches = 0
    
    perm = tn.tensor(np.random.permutation(inputs_train.shape[0]))
    perms = tn.tensor_split(perm,inputs_train.shape[0]//batch_size)
        
    for p in perms:
        #tme = datetime.datetime.now() - tme
        #print('t0',tme)
        # tme = datetime.datetime.now()
        inputs = inputs_train[p,:,:,:].to(device_name)
        labels = labels_train[p].to(device_name)

        # tme = datetime.datetime.now() - tme
        # print('t1',tme)
        
        # tme = datetime.datetime.now()
        inputs = tn.reshape(inputs,[-1,3]+2*N_shape)
        # tme = datetime.datetime.now() - tme
        # print('t2',tme)
        
        # tme = datetime.datetime.now()
        optimizer.zero_grad()
        # Make predictions for this batch
        outputs = model(inputs)
        # Compute the loss and its gradients
        loss = loss_function(outputs, labels)
        
        # regularization
        l2_lambda = 0.005
        l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
        loss = loss+l2_lambda*l2_norm
        
        loss.backward()
        # Adjust learning weights
        optimizer.step()
        # tme = datetime.datetime.now() - tme
        # print('t3',tme)
        n_correct += tn.sum(tn.max(outputs,1)[1] == labels).cpu()   
        n_total+=inputs.shape[0]
        
        loss_total += loss.item()
        n_batches+=1
        # print('\t\tbatch %d error %e'%(k+1,loss))
        #tme = datetime.datetime.now()
        
    return loss_total/n_batches, n_correct/n_total



def test_data():
    n_total = 0 
    n_correct = 0
    loss_total = 0
    
    perm = tn.tensor(np.random.permutation(inputs_test.shape[0]))
    perms = tn.tensor_split(perm,inputs_test.shape[0]//batch_size)
        
        
    for p in perms:
        inputs = inputs_test[p,:,:,:].to(device_name)
        labels = labels_test[p].to(device_name)
        inputs = tn.reshape(inputs,[-1,3]+2*N_shape)
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss_total += loss.item()
        n_correct += tn.sum(tn.max(outputs,1)[1] == labels)   
        n_total+=inputs.shape[0]
        
    return loss_total/len(dataloader_test), n_correct/n_total



In [7]:
n_epochs = 200

history_test_accuracy = []
history_test_loss = []
history_train_accuracy = []
history_train_loss = []

for epoch in range(n_epochs):
    print('Epoch %d/%d'%(epoch+1,n_epochs))
    
    time_epoch = datetime.datetime.now()
    
    model.train(True)
    train_loss, train_acc = do_epoch(epoch)
    model.train(False)
    
    test_loss, test_acc = test_data()
    scheduler.step()
    
    time_epoch = datetime.datetime.now() - time_epoch
    
    print('\tTraining loss %e training accuracy %5.4f test loss %e test accuracy %5.4f'%(train_loss,train_acc,test_loss,test_acc))
    print('\tTime for the epoch',time_epoch)
    history_test_accuracy.append(test_acc)
    history_test_loss.append(test_loss)
    history_train_accuracy.append(train_acc)
    history_train_loss.append(train_loss)
    


Epoch 1/200
	Training loss 4.088508e+00 training accuracy 0.1746 test loss 1.665319e+02 test accuracy 0.1713
	Time for the epoch 0:00:15.883593
Epoch 2/200
	Training loss 3.882070e+00 training accuracy 0.1765 test loss 1.665091e+02 test accuracy 0.1750
	Time for the epoch 0:00:14.362440
Epoch 3/200
	Training loss 3.705994e+00 training accuracy 0.1785 test loss 1.664987e+02 test accuracy 0.1750
	Time for the epoch 0:00:14.248921
Epoch 4/200
	Training loss 3.545270e+00 training accuracy 0.1790 test loss 1.664954e+02 test accuracy 0.1750
	Time for the epoch 0:00:14.853008
Epoch 5/200
	Training loss 3.397970e+00 training accuracy 0.1790 test loss 1.664972e+02 test accuracy 0.1750
	Time for the epoch 0:00:14.744421
Epoch 6/200
	Training loss 3.263061e+00 training accuracy 0.1790 test loss 1.664878e+02 test accuracy 0.1750
	Time for the epoch 0:00:15.099879
Epoch 7/200
	Training loss 3.139448e+00 training accuracy 0.1790 test loss 1.664867e+02 test accuracy 0.1750
	Time for the epoch 0:00:14

KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(np.arange(len(history_train_accuracy))+1,np.array(history_train_accuracy))
plt.plot(np.arange(len(history_test_accuracy))+1,np.array(history_test_accuracy))
plt.legend(['training','test'])

plt.figure()
plt.plot(np.arange(len(history_train_loss))+1,np.array(history_train_loss))
plt.plot(np.arange(len(history_test_loss))+1,np.array(history_test_loss))
plt.legend(['training','test'])
    

In [ ]:
inputs_train.shape[0]//batch_size
        